# 制作命令行工具并测试


C语言是和系统最近的编程语言之一,而很多系统工具也是C写的.


## 制作小工具


在linux的世界里,很多工作都是由小工具完成的,它恐怕是现如今流行的`微服务`架构最早的实践了.

所谓小工具有这样的特点:

1. 从标准输入读取数据
2. 在标准输出显示数据
3. 处理文本数据，而不是难以阅读的二进制格式
4. 只做一件简单的事

通过标准输入输出可以方便的做重定向,而文本数据除了机器好读,人也好读.


所谓重新定向是指使用`>`,`>>`作为输出,使用`<`,`<<`作为输入


比较让人熟知的就是`grep`小工具了,它的作用是过滤掉文本中不含关键字的行.而一般我们都是拿他和其他工具配合使用的,比如我希望查看之前使用过的带`python`的历史记录,那就可以这样写:

```shell
history | grep python
```

当然了其他语言比如python也可以制作小工具,而且由于语法更加简单,抽象层次更高,实际上写起来会更加简单,但在这边用c/c++来做小工具有其合理的一面

1. 不用依赖环境

    c/c++不依赖于运行环境,只要编译通过了就可以运行二进制代码.而带有vm的语言比如java,python,脱离了vm就无法运行了
    
2. 高效

    多数情况下,,相同的逻辑,C/C++制作的工具有最高的运行效率
    
3. 小巧

    像go语言,也是编译成二进制后执行的语言,但由于其抽象层次更高(虽然也没高多少)而且有个运行时,生成的代码比使用c/c++写的大得多,而c++如果使用模板的话也会比C写的大得多

## 从一个例子开始

我们来写一个gps数据格式转换工具,使用它我们可以将gps数据转化为符合规定的json格式,gps数据是形如:

```
42.363400,-71.098465,Speed = 21
42.363400,-71.097588,Speed = 23
42.363400,-71.098465,Speed = 27
.
.
.
```
的csv数据

而地图应用需要的格式则形如:

```json
data = [
{latitude: 42.363400, longitude: -71.098465, info: 'Speed = 21'},
.
.
.
]
```
的json数据



好,开始我们的小程序:

In [2]:
%%writefile ./code/create_cmdline_tool/geo2json/source/src/geo2json.c
#include <stdio.h>

int main(){
    float latitude;
    float longitude;
    char info[80];
    int started = 0;
    puts("data=[");
    while (scanf("%f,%f,%79[^\n]",&latitude,&longitude,&info) == 3){
        if (started){
            printf(",\n");
        } else {
            started = 1;
        }
        printf("{latitude: %f, longitude: %f, info: '%s'}",latitude,longitude,info);
    }
    puts("\n]");
    return 0;
}

Overwriting ./code/create_cmdline_tool/geo2json/source/src/geo2json.c


In [3]:
%%writefile ./code/create_cmdline_tool/geo2json/CMakeLists.txt
#项目编译环境
cmake_minimum_required (VERSION 2.8)
project (geo2json)

# 编译连接生成demo
set(CMAKE_RUNTIME_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/build/bin)
aux_source_directory(${CMAKE_BINARY_DIR}/source/src DIR_SRCS)
add_executable(geo2json ${DIR_SRCS})

Writing ./code/create_cmdline_tool/geo2json/CMakeLists.txt


In [4]:
!cmake ./code/create_cmdline_tool/geo2json/CMakeLists.txt

-- The C compiler identification is AppleClang 10.0.0.10001044
-- The CXX compiler identification is AppleClang 10.0.0.10001044
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /Library/Developer/CommandLineTools/usr/bin/c++
-- Check for working CXX compiler: /Library/Developer/CommandLineTools/usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/create_cmdline_tool/geo2json


In [5]:
!make -C ./code/create_cmdline_tool/geo2json/

Scanning dependencies of target geo2json
[ 50%] Building C object CMakeFiles/geo2json.dir/source/src/geo2json.c.o
/Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/create_cmdline_tool/geo2json/source/src/geo2json.c:9:56: warning: 
      format specifies type 'char *' but the argument has type 'char (*)[80]'
      [-Wformat]
    while (scanf("%f,%f,%79[^\n]",&latitude,&longitude,&info) == 3){
                        ~~~~~~                         ^~~~~
1 warning generated.
[100%] Linking C executable build/bin/geo2json
[100%] Built target geo2json


In [7]:
!./code/create_cmdline_tool/geo2json/build/bin/geo2json \
< ./code/create_cmdline_tool/geo2json/data/gpsdata.csv \
> ./code/create_cmdline_tool/geo2json/out/gpsdata.json

In [8]:
!cat ./code/create_cmdline_tool/geo2json/out/gpsdata.json

data=[
{latitude: 42.363400, longitude: -71.098465, info: 'Speed = 21'},
{latitude: 42.363327, longitude: -71.097588, info: 'Speed = 23'},
{latitude: 42.363255, longitude: -71.096710, info: 'Speed = 17'},
{latitude: 42.363182, longitude: -71.095833, info: 'Speed = 22'},
{latitude: 42.363110, longitude: -71.094955, info: 'Speed = 14'},
{latitude: 42.363037, longitude: -71.094078, info: 'Speed = 16'},
{latitude: 42.362965, longitude: -71.093201, info: 'Speed = 18'},
{latitude: 42.362892, longitude: -71.092323, info: 'Speed = 22'},
{latitude: 42.362820, longitude: -71.091446, info: 'Speed = 17'},
{latitude: 42.362747, longitude: -71.090569, info: 'Speed = 23'},
{latitude: 42.362675, longitude: -71.089691, info: 'Speed = 14'},
{latitude: 42.362602, longitude: -71.088814, info: 'Speed = 19'},
{latitude: 42.362530, longitude: -71.087936, info: 'Speed = 16'},
{latitude: 42.362457, longitude: -71.087059, info: 'Speed = 16'},
{latitude: 423.623840, longitude: -71.086182, info: 'Speed = 21'}
]


### 数据验证

不难发现,输入数据中有错误数据,我们的程序要加入一些验证逻辑来进行错误检验,当然了,错误信息应该输出在stderr里

In [9]:
%%writefile ./code/create_cmdline_tool/geo2json/source/src/geo2json.c
#include <stdio.h>

int main(){
    float latitude;
    float longitude;
    char info[80];
    int started = 0;
    puts("data=[");
    while (scanf("%f,%f,%79[^\n]",&latitude,&longitude,&info) == 3){
        if (started){
            printf(",\n");
        } else {
            started = 1;
        }
        
        if ((latitude < -90.0) || (latitude > 90.0)) {
             fprintf(stderr,"Invalid latitude: %f\n", latitude);
             continue;
        }
        if ((longitude < -180.0) || (longitude > 180.0)) {
             fprintf(stderr, "Invalid longitude: %f\n", longitude);
             continue;
        }
        
        printf("{latitude: %f, longitude: %f, info: '%s'}",latitude,longitude,info);
    }
    puts("\n]");
    return 0;
}

Overwriting ./code/create_cmdline_tool/geo2json/source/src/geo2json.c


In [10]:
!make -C ./code/create_cmdline_tool/geo2json/

Scanning dependencies of target geo2json
[ 50%] Building C object CMakeFiles/geo2json.dir/source/src/geo2json.c.o
/Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/create_cmdline_tool/geo2json/source/src/geo2json.c:9:56: warning: 
      format specifies type 'char *' but the argument has type 'char (*)[80]'
      [-Wformat]
    while (scanf("%f,%f,%79[^\n]",&latitude,&longitude,&info) == 3){
                        ~~~~~~                         ^~~~~
1 warning generated.
[100%] Linking C executable build/bin/geo2json
[100%] Built target geo2json


In [11]:
!./code/create_cmdline_tool/geo2json/build/bin/geo2json \
< ./code/create_cmdline_tool/geo2json/data/gpsdata.csv \
> ./code/create_cmdline_tool/geo2json/out/gpsdata.json

Invalid latitude: 423.623840


In [12]:
!cat ./code/create_cmdline_tool/geo2json/out/gpsdata.json

data=[
{latitude: 42.363400, longitude: -71.098465, info: 'Speed = 21'},
{latitude: 42.363327, longitude: -71.097588, info: 'Speed = 23'},
{latitude: 42.363255, longitude: -71.096710, info: 'Speed = 17'},
{latitude: 42.363182, longitude: -71.095833, info: 'Speed = 22'},
{latitude: 42.363110, longitude: -71.094955, info: 'Speed = 14'},
{latitude: 42.363037, longitude: -71.094078, info: 'Speed = 16'},
{latitude: 42.362965, longitude: -71.093201, info: 'Speed = 18'},
{latitude: 42.362892, longitude: -71.092323, info: 'Speed = 22'},
{latitude: 42.362820, longitude: -71.091446, info: 'Speed = 17'},
{latitude: 42.362747, longitude: -71.090569, info: 'Speed = 23'},
{latitude: 42.362675, longitude: -71.089691, info: 'Speed = 14'},
{latitude: 42.362602, longitude: -71.088814, info: 'Speed = 19'},
{latitude: 42.362530, longitude: -71.087936, info: 'Speed = 16'},
{latitude: 42.362457, longitude: -71.087059, info: 'Speed = 16'},

]


## 用C构建带参数的命令行工具

`unistd.h`是C语言的一个准标准库,它是linux,mac osx等类unix系统的自带的库,但windows并不支持.要在windows下使用需要自己写一个头文件,如下:

```cpp
/** This file is part of the Mingw32 package. 
*  unistd.h maps     (roughly) to io.h 
*/  
#ifndef _UNISTD_H  
#define _UNISTD_H  
#include <io.h>  
#include <process.h>  
#endif /* _UNISTD_H */ 

```


使用它便利的构建命令行工具`getopt`

`getopt()`用来分析命令行参数.参数`argc`和`argv`是由`main()`传递的参数个数和内容.参数`optstring`则代表欲处理的选项字符串.此函数会返回在`argv`中下一个的选项字母,此字母会对应参数`optstring`中的字母.如果选项字符串里的字母后接着冒号":".则表示还有相关的参数,全域变量`optarg` 即会指向此额外参数.如果`getopt()`找不到符合的参数则会印出错信息,并将全域变量`optopt`设为"?"字符,如果不希望`getopt()`印出错信息,则只要将全域变量`opterr`设为0即可.

### 短参数的定义

`getopt()`使用`optstring`所指的字串作为短参数列表,像`"1ac:d::"`就是一个短参数列表.短参数的定义是一个`'-'`后面跟一个字母或数字,象`-a`, `-b`就是一个短参数.每个数字或字母定义一个参数.

其中短参数在`getopt`定义里分为三种：

+ 不带值的参数,它的定义即是参数本身
+ 必须带值的参数,它的定义是在参数本身后面再加一个冒号
+ 可选值的参数,它的定义是在参数本身后面加两个冒号

在这里拿上面的`"1ac:d::"`作为样例进行说明,其中的
+ `1`,`a`就是不带值的参数
+ `c`是必须带值的参数
+ `d`是可选值的参数

在实际调用中，`'-1 -a -c cvalue -d'`, `'-1 -a -c cvalue -ddvalue'`, `'-1a -ddvalue -c cvalue'`都是合法的.
这里需要注意三点:

+ 不带值的参数可以连写,像`1`和`a`是不带值的参数,它们可以`-1` `-a`分开写,也可以`-1a`或`-a1`连写.
+ 参数不分先后顺序,`'-1a -c cvalue -ddvalue'`和`'-d -c cvalue -a1'`的解析结果是一样的.
+ 要注意可选值的参数的值与参数之间不能有空格,必须写成`-ddvalue`这样的格式,如果写成`-d dvalue`这样的格式就会解析错误.

### 返回值

`getopt()`每次调用会逐次返回命令行传入的参数. 　 
+ 当没有参数的最后的一次调用时,`getopt()`将返回-1.　 
+ 当解析到一个不在`optstring`里面的参数，或者一个必选值参数不带值时,返回'?'.　 
+ 当`optstring`是以':'开头时,缺值参数的情况下会返回':',而不是'?'.

In [13]:
%%writefile ./code/create_cmdline_tool/sqrt_opt/source/src/sqrt_opt.c
#include <stdlib.h>
#include <stdio.h>
#include <unistd.h>
#include <math.h>
const char * version = "0.1.0";

int main(int argc, char **argv)  {  
    int ch; 
    double result;
    opterr = 0;
    while ((ch = getopt(argc,argv,"s:v\n"))!=-1){  
        switch(ch){  
            case 'v':
                printf("option v: %s\n",version);
                break;  
            case 's':
                result = sqrt(atoi(optarg));
                printf("%f\n",result);
                break;
            default:
                printf("wrong args\n");
        }  
    }  
    return 0;
}  

Writing ./code/create_cmdline_tool/sqrt_opt/source/src/sqrt_opt.c


In [18]:
%%writefile ./code/create_cmdline_tool/sqrt_opt/CMakeLists.txt
#项目编译环境
cmake_minimum_required (VERSION 2.8)
project (sqrt_opt)

# 编译连接生成demo
set(CMAKE_RUNTIME_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/build/bin)
aux_source_directory(${CMAKE_BINARY_DIR}/source/src DIR_SRCS)
add_executable(sqrt_opt ${DIR_SRCS})

Overwriting ./code/create_cmdline_tool/sqrt_opt/CMakeLists.txt


In [19]:
!cmake ./code/create_cmdline_tool/sqrt_opt/CMakeLists.txt

-- Configuring done
-- Generating done
-- Build files have been written to: /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/create_cmdline_tool/sqrt_opt


In [20]:
!make -C ./code/create_cmdline_tool/sqrt_opt/

Scanning dependencies of target sqrt_opt
[ 50%] Building C object CMakeFiles/sqrt_opt.dir/source/src/sqrt_opt.c.o
[100%] Linking C executable build/bin/sqrt_opt
[100%] Built target sqrt_opt


In [21]:
!./code/create_cmdline_tool/sqrt_opt/build/bin/sqrt_opt -v

option v: 0.1.0


In [23]:
!./code/create_cmdline_tool/sqrt_opt/build/bin/sqrt_opt -s36

6.000000


### 使用cmake test 测试命令行工具

cmake自带一个可以测试命令行工具的方案,我们来修改我们的`CMakeLists.txt`,测试的形式:

```
add_test (<test_case_name> ${CMAKE_BINARY_DIR}/build/bin/sqrt_opt -s36)
set_tests_properties (<test_case_name>
  PROPERTIES PASS_REGULAR_EXPRESSION "6.0")
```

`add_test`用于增加一条测试,`set_tests_properties`用于验证测试是否符合预期.使用`PASS_REGULAR_EXPRESSION`利用正则匹配来验证是否结果符合预期.


In [24]:
%%writefile ./code/create_cmdline_tool/sqrt_opt/CMakeLists.txt
#项目编译环境
cmake_minimum_required (VERSION 2.8)
project (sqrt_opt)

# 编译连接生成demo
set(CMAKE_RUNTIME_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/build/bin)
aux_source_directory(${CMAKE_BINARY_DIR}/source/src DIR_SRCS)
add_executable(sqrt_opt ${DIR_SRCS})
#测试
enable_testing()
add_test (TEST_VERSION ${CMAKE_BINARY_DIR}/build/bin/sqrt_opt -v)
set_tests_properties (TEST_VERSION
  PROPERTIES PASS_REGULAR_EXPRESSION "v: 0.1.0")
add_test (TEST_36 ${CMAKE_BINARY_DIR}/build/bin/sqrt_opt -s36)
set_tests_properties (TEST_36
  PROPERTIES PASS_REGULAR_EXPRESSION "6.0")
add_test (TEST_2 ${CMAKE_BINARY_DIR}/build/bin/sqrt_opt -s2)
set_tests_properties (TEST_2
  PROPERTIES PASS_REGULAR_EXPRESSION "1.41")

Overwriting ./code/create_cmdline_tool/sqrt_opt/CMakeLists.txt


In [25]:
!cmake ./code/create_cmdline_tool/sqrt_opt/CMakeLists.txt

-- Configuring done
-- Generating done
-- Build files have been written to: /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/create_cmdline_tool/sqrt_opt


In [26]:
!make -C ./code/create_cmdline_tool/sqrt_opt/

[100%] Built target sqrt_opt


In [27]:
!make -C ./code/create_cmdline_tool/sqrt_opt/ test

Running tests...
Test project /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/create_cmdline_tool/sqrt_opt
    Start 1: TEST_VERSION
1/3 Test #1: TEST_VERSION .....................   Passed    0.01 sec
    Start 2: TEST_36
2/3 Test #2: TEST_36 ..........................   Passed    0.01 sec
    Start 3: TEST_2
3/3 Test #3: TEST_2 ...........................   Passed    0.01 sec

100% tests passed, 0 tests failed out of 3

Total Test time (real) =   0.03 sec
